# Introducción

En este notebook se crearán los videos correspondientes a las acciones 'Severity' con el objetivo de compensar el desequilibrio del conjunto de datos de entrenamiento.


In [ ]:
import json
import os
from collections import defaultdict
import cv2
import numpy as np
from sklearn.utils import resample
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import vidaug.augmentors as va
import random
from PIL import ImageOps, Image



In [ ]:
#Funcion de lectura de las etiquetas a partir de un json
def load_foul_labels(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    action_labels = {}
    for action_id, action_data in data['Actions'].items():
        label = action_data['Severity']  # Extraer la etiqueta de 'Severity'
        action_name = f"action_{action_id}"  # Formar el nombre de la acción
        action_labels[action_name] = label
    
    return action_labels

In [ ]:
# Carga de etiquetas en un listado ordenado
ls = load_foul_labels("F:/data/mvfouls/train/annotations.json")
ls = sorted(ls.items())
sorted_labels_train = sorted(ls, key=lambda x: int(x[0].split('_')[1]))


In [ ]:
#Funcion para agrupar la ruta de los videos por accion  

def group_videos_by_action(root_path, video_extensions=None):
    if video_extensions is None:
        video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv']

    grouped_videos = defaultdict(list)

    for dirpath, dirnames, filenames in os.walk(root_path):
        for file in filenames:
            if any(file.lower().endswith(ext) for ext in video_extensions):
                action_name = os.path.basename(dirpath)  # Nombre de la carpeta contenedora
                file_path = os.path.join(dirpath, file)
                grouped_videos[action_name].append(file_path)
    
    sorted_actions = sorted(
        grouped_videos.items(), 
        key=lambda x: int(x[0].split('_')[1])  # Extraer número después de "action_"
    )
    return grouped_videos

root_path = "F:/data/mvfouls/train"
videos_path = group_videos_by_action(root_path = root_path)


In [ ]:
sorted_labels_train[:10]

In [ ]:
import pandas as pd
from collections import Counter


# Convertir a DataFrame
df = pd.DataFrame(sorted_labels_train, columns=['action', 'label'])

# Contar la frecuencia de cada clase
class_counts = Counter(df['label'])

# Mostrar la distribución de clases
print("Distribución de clases:")
print(class_counts)

Agrupamos y convertimos las etiquetas según su severidad

In [ ]:
# Diccionario de mapeo
conversion = {
    "1.0": "no card",
    "2.0": "no card",
    "3.0": "yellow card",
    "4.0": "yellow card",
    "5.0": "red card",
    "": ""  # Estas son No offence no queremos tratar con ellas
}

# guardamos la conversion
updated_labels = [(action, conversion[label]) for action, label in sorted_labels_train]

print(updated_labels[:10])




In [ ]:
#Funciones para cargar los videos y guardar los nuevos creados a partir de la función de data augmentation.

def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return np.array(frames)

def save_video(frames, output_path, fps=30):
    height, width, layers = frames[0].shape
    size = (width, height)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    for frame in frames:
        out.write(frame)
    out.release()

#Función que identifica las acciones con etiqueta 5.0 y crea una lista aleatoria de acciones para editar   
def identify_and_resample_severity(actions, labels, target_count):
    no_offence_actions = [action for action, label in labels.items() if label == '5.0']
    resampled_actions = resample(no_offence_actions, replace=True, n_samples=target_count, random_state=123)
    return resampled_actions




In [ ]:
base_path = 'F:/data/mvfouls/train'
actions= videos_path
actions_labels_dict = {action: label for action, label in sorted_labels_train}
target_count = 200 #Numero objetivo de acciones "No offence" después de resamplear
resampled_actions = identify_and_resample_severity(actions, actions_labels_dict, target_count)

# Data Augmentation para Videos Etiquetados como 'No offence'

Este código implementa un conjunto de funciones para realizar **data augmentation** en videos etiquetados como `'No offence'`, con el propósito de equilibrar un conjunto de datos de entrenamiento. A continuación, se detallan las transformaciones aplicadas y el flujo de procesamiento.

## Transformaciones

### Transformaciones básicas con `vidaug`
- **Rotación horizontal**: Voltea los frames horizontalmente para simular diferentes perspectivas.

### Transformaciones adicionales
- **`adjust_saturation`**: Ajusta la saturación de los frames para variar la intensidad de los colores.
- **`adjust_brightness`**: Modifica el brillo de los frames para simular cambios en la iluminación.
- **`adjust_contrast`**: Ajusta el contraste para resaltar o suavizar detalles en los frames.
- **`add_random_noise`**: Introduce ruido aleatorio para simular imperfecciones o condiciones de captura.
- **`apply_gaussian_blur`**: Aplica un desenfoque gaussiano para suavizar los frames.
- **`random_rotation`**: Rota los frames con un ángulo aleatorio dentro de un rango máximo.
- **`add_colored_dots`**: Añade puntos semi-transparentes de colores (rojo, negro y blanco) para aumentar la diversidad visual.
- **`shear`**: Aplica una transformación de cizallamiento para distorsionar los frames.
- **`apply_vignette`**: Agrega un efecto de viñeta para oscurecer los bordes de los frames.
- **`grayscale`**: Convierte los frames a escala de grises, manteniendo el formato BGR.
- **`edge_enhance`**: Resalta los bordes de los frames mediante un filtro de detección de bordes.

## Aplicación en `augment_video`

La función `augment_video` recibe una lista de frames y aplica un subconjunto de transformaciones seleccionadas de forma aleatoria o todas, si no se especifica. Combina las transformaciones de `vidaug` con las funciones adicionales para generar videos con mayor diversidad, ideales para mejorar el rendimiento de modelos de machine learning.

## Procesamiento y guardado de videos

La función `navigate_and_resample_videos` itera sobre una lista de acciones remuestreadas, carga los videos asociados desde una ruta base, aplica un número configurable de transformaciones aleatorias (por defecto, 4) y guarda los videos aumentados en una carpeta específica (`{action}_augmented`). Si la carpeta ya existe, se crea una nueva con un sufijo numérico para evitar conflictos.

### Funciones auxiliares
- **`load_video`**: Carga los frames de un video desde un archivo.
- **`save_video`**: Guarda una lista de frames como un nuevo video en formato MP4.

## Flujo general
1. Se identifican las acciones etiquetadas como `'No offence'` y se remuestrean para alcanzar un número objetivo.
2. Para cada acción, se seleccionan aleatoriamente un número fijo de transformaciones.
3. Los videos correspondientes se cargan, transforman con `augment_video` y se guardan en una nueva carpeta.

Este enfoque permite generar un conjunto de datos más diverso y equilibrado, adecuado para tareas de aprendizaje automático.

In [ ]:
import os
import shutil
import numpy as np
import cv2
import vidaug.augmentors as va
import random

seq = va.Sequential([
    va.HorizontalFlip(),
])

def adjust_saturation(image, factor=1.5):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[..., 1] = np.clip(hsv[..., 1] * factor, 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

def adjust_brightness(image, factor=1.2):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[..., 2] = np.clip(hsv[..., 2] * factor, 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

def adjust_contrast(image, factor=1.3):
    img_float = image.astype(np.float32)
    mean = np.mean(img_float, axis=(0, 1))
    contrasted = np.clip((img_float - mean) * factor + mean, 0, 255)
    return contrasted.astype(np.uint8)

def add_random_noise(image, intensity=15):
    noise = np.random.randint(-intensity, intensity, image.shape, dtype=np.int16)
    noisy_image = np.clip(image.astype(np.int16) + noise, 0, 255)
    return noisy_image.astype(np.uint8)

def apply_gaussian_blur(image, ksize=(3, 3)):
    return cv2.GaussianBlur(image, ksize, 0)

def random_rotation(frames, max_angle=10):
    angle = random.uniform(-max_angle, max_angle)
    rotated_frames = []
    for frame in frames:
        h, w = frame.shape[:2]
        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
        rotated = cv2.warpAffine(frame, M, (w, h), borderMode=cv2.BORDER_REFLECT)
        rotated_frames.append(rotated)
    return rotated_frames

def add_colored_dots(image, num_dots=500, dot_size=2, alpha=0.6):
    overlay = image.copy()
    h, w = image.shape[:2]
    colors = [(0, 0, 255), (0, 0, 0), (255, 255, 255)]
    for _ in range(num_dots):
        x, y = random.randint(0, w - 1), random.randint(0, h - 1)
        color = random.choice(colors)
        cv2.circle(overlay, (x, y), dot_size, color, -1)
    return cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)

def shear(image, max_shear=0.05):
    rows, cols = image.shape[:2]
    M = np.float32([[1, random.uniform(-max_shear, max_shear), 0], 
                    [random.uniform(-max_shear, max_shear), 1, 0]])
    sheared_image = cv2.warpAffine(image, M, (cols, rows))
    return sheared_image

def apply_vignette(image, intensity=0.6):
    h, w = image.shape[:2]
    kernel_x = cv2.getGaussianKernel(w, w / 2)
    kernel_y = cv2.getGaussianKernel(h, h / 2)
    kernel = kernel_y * kernel_x.T
    mask = 255 * kernel / np.max(kernel)
    mask = cv2.normalize(mask, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    vignette = np.copy(image)
    for i in range(3):
        vignette[:, :, i] = vignette[:, :, i] * (1 - intensity * (1 - mask))
    return vignette.astype(np.uint8)

def grayscale(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

def edge_enhance(image):
    kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
    enhanced = cv2.filter2D(image, -1, kernel)
    return np.clip(enhanced, 0, 255).astype(np.uint8)

def augment_video(frames, apply_transforms=None):
    all_transforms = [
        "vidaug",
        "saturation",
        "brightness",
        "contrast",
        "noise",
        "blur",
        "rotation",
        "colored_dots",
        "shear",
        "vignette",
        "grayscale",
        "edge_enhance"
    ]
    
    if apply_transforms is None:
        apply_transforms = all_transforms
    
    transformed_frames = frames.copy()
    
    for transform in apply_transforms:
        if transform == "vidaug":
            transformed_frames = seq(transformed_frames)
        elif transform == "saturation":
            transformed_frames = [adjust_saturation(frame) for frame in transformed_frames]
        elif transform == "brightness":
            transformed_frames = [adjust_brightness(frame) for frame in transformed_frames]
        elif transform == "contrast":
            transformed_frames = [adjust_contrast(frame) for frame in transformed_frames]
        elif transform == "noise":
            transformed_frames = [add_random_noise(frame) for frame in transformed_frames]
        elif transform == "blur":
            transformed_frames = [apply_gaussian_blur(frame) for frame in transformed_frames]
        elif transform == "rotation":
            transformed_frames = random_rotation(transformed_frames)
        elif transform == "colored_dots":
            transformed_frames = [add_colored_dots(frame) for frame in transformed_frames]
        elif transform == "shear":
            transformed_frames = [shear(frame) for frame in transformed_frames]
        elif transform == "vignette":
            transformed_frames = [apply_vignette(frame) for frame in transformed_frames]
        elif transform == "grayscale":
            transformed_frames = [grayscale(frame) for frame in transformed_frames]
        elif transform == "edge_enhance":
            transformed_frames = [edge_enhance(frame) for frame in transformed_frames]
    
    if len(transformed_frames) != len(frames):
        transformed_frames = transformed_frames[:len(frames)]
    
    return np.array(transformed_frames)

def navigate_and_resample_videos(base_path, actions, resampled_actions, num_transforms=4):
    all_transforms = [
        "vidaug",
        "saturation",
        "brightness",
        "contrast",
        "noise",
        "blur",
        "rotation",
        "colored_dots",
        "shear",
        "vignette",
        "grayscale",
        "edge_enhance"
    ]

    for action in resampled_actions:
        if action in actions:
            video_list = actions[action]
            selected_transforms = random.sample(all_transforms, min(num_transforms, len(all_transforms)))
            print(f"Transformaciones para la acción '{action}': {selected_transforms}")
            
            base_folder_name = f"{action}_augmented"
            new_action_path = os.path.join(base_path, base_folder_name)
            suffix = 1
            while os.path.exists(new_action_path):
                new_action_path = os.path.join(base_path, f"{base_folder_name}_{suffix}")
                suffix += 1
            os.makedirs(new_action_path)

            for video_file in video_list:
                video_path = os.path.join(base_path, video_file)
                frames = load_video(video_path)
                if len(frames) == 0:
                    continue

                augmented_frames = augment_video(frames, apply_transforms=selected_transforms)
                
                video_name = os.path.splitext(os.path.basename(video_file))[0]
                output_video_path = os.path.join(new_action_path, f"{video_name}_augmented.mp4")
                print(f"Guardando clip: {output_video_path}")
                save_video(augmented_frames, output_video_path)

In [ ]:
navigate_and_resample_videos(base_path, actions, resampled_actions)

In [ ]:
#Función para renombrar las carpetas de videos aumentados siguiendo el orden de acciones
def rename_action_folders_sequential(base_path, start_number):
    counter = start_number
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        
        # Verificamos si es una carpeta y si contiene 'augmented' en el nombre
        if os.path.isdir(folder_path) and 'augmented' in folder_name:
            new_folder_name = f"action_{counter}"
            new_folder_path = os.path.join(base_path, new_folder_name)
            print(f"Renombrando {folder_path} a {new_folder_path}")
            os.rename(folder_path, new_folder_path)
            
            counter += 1 
    
    print("Renombrado secuencial completo.")

# Ruta base 
base_path = 'F:/data/mvfouls/train'

start_number = 3853  # Acción por la que empezar a renombrar
rename_action_folders_sequential(base_path, start_number)


# EN EL 3175 empiezan las 5.0

In [ ]:
#AÑADIMOS NUEVAS ACCIONES AL DICCIONARIO DE LABELS 

# Calcular cuántas nuevas acciones necesitamos para llegar a 3345
new_actions_needed = 3174 - len(sorted_labels_train)+1

# Generar nuevas acciones con la etiqueta 'No Offence'
new_actions = [('action_' + str(i), '3.0') for i in range(len(sorted_labels_train), len(sorted_labels_train) + new_actions_needed)]

# Agregar las nuevas acciones al diccionario
updated_labels = sorted_labels_train + new_actions

# Verificar el total final y mostrar algunas de las nuevas acciones
print(f"Total de acciones después de la ampliación: {len(updated_labels)}")
print(updated_labels[:])  # Mostrar las últimas 10 acciones para verificar


In [ ]:
# Calcular cuántas nuevas acciones necesitamos para llegar a 3345
new_actions_needed = 4052 - len(updated_labels)+1

# Crear las nuevas acciones con etiqueta '5.0' desde el último índice actual
start_index = int(updated_labels[-1][0].split('_')[1]) + 1  # último índice + 1

# Generar nuevas acciones
new_actions_5_0 = [('action_' + str(i), '5.0') for i in range(start_index, start_index + new_actions_needed)]

# Agregar al listado actualizado
updated_labels += new_actions_5_0

# Verificación
print(f"Total final de acciones: {len(updated_labels)}")
print("Últimas acciones añadidas:")
print(updated_labels[-new_actions_needed:])


In [ ]:
3193- 3174

In [ ]:
updated_labels[2917:3100]